In [ ]:
using DrWatson
@quickactivate "Simulation"
include(srcdir("Simulation.jl"))

import .Simulation as Sim
using ProgressMeter

Threads.nthreads()

In [ ]:
params = Sim.VarPar(
    max_ar=520,
    error=1e-7,
    t_coeff=0.55,
    temp=1e-7,
    voltage=range(0.05, 3.50, step=0.025),
    w=(-15.0, 15.0)
)
currents::Vector{Real} = zeros(length(params.voltage))

# check max_MAR
Sim.num_MAR(first(params.voltage), params)

In [ ]:
function simpsons_rule(f, a, b, n)
    h = (b - a) / n
    k = 0.0
    x = a + h
    for i in range(1,n/2+1)
        k += 2*f(x)
        x += 2*h
    end
    x = a+ 2*h
    for i in range(1,n/2)
        k += 2*f(x)
        x += 2*h
    end
    return (h/3)*(f(a) + f(b) + k)
end
f(x) = Sim.integrand(params, x, 0.05, 47, 520)

# Usage
a, b, n = -15, 15.0, 1000  # n must be even
println("Simpson's Rule Result = ", simpsons_rule(f, a, b, n))

# x = Sim.integrand(params, 2, 0.05, 47, 520)

In [ ]:

for v in params.voltage
    x = Sim.∫(params, v, Sim.num_MAR(v, params), 520)
    println("Voltage: ", v, " Current: ", x)
end

In [ ]:
progress = Progress(length(params.voltage), 1, "Processing ...")
Threads.@threads for i in 1:length(params.voltage)
    curr_v::Real = params.voltage[i]
    req_MAR::Int = Sim.num_MAR(curr_v, params)
    currents[i] = Real(Sim.∫(params, curr_v, req_MAR, 520))
    next!(progress)
end

In [ ]:
# norm_currents = currents ./ (Sim.G_0*params.t_coeff);
norm_currents = currents;

In [ ]:
open(datadir("D-T$(params.t_coeff).dat"), "w") do io
    for i in 1:length(params.voltage)
        println(io, params.voltage[i], "\t", currents[i])
    end
end
println("Data saved to $(datadir("D-T$(params.t_coeff).dat"))")

In [ ]:
function plot_file(s)
    ax = []
    curve = []
    open(datadir(s),"r") do file
        for (i, line) in enumerate(eachline(file))
            line = strip(line)
            line = replace(line, "\t"=>" ")
            line = split(line, " ")
            filter!(x -> x != "", line)
            if length(line) < 2
                continue
            end
            x, y = line
            if y == "nan"
                continue
            end
            x = parse(Float64, x)
            y = parse(Float64, y)
            push!(ax, x)
            push!(curve, y)
        end
    end
    plot!(
        ax,
        curve
    )
    # xlims!(0,3.5)
    # ylims!(0, 6)
    return plot!()
end

In [ ]:
using Plots
# files = [
#     "0.125",
#     "0.55",
#     "0.75",
#     "0.95",
#     "0.99"
# ]

# for file in files
#     plot_file("D-T$(file).dat")
# end

plot_file("D-T$(params.t_coeff).dat")